In [3]:
pip install opencv-python opencv-python-headless numpy

Note: you may need to restart the kernel to use updated packages.


In [5]:
import cv2
import os

# Input and output directories
input_dir = "E:/Research/real/Extracted_Frames_Angry"  # Directory with extracted frames
output_dir = "E:/Research/real/Cropped_Faces"  # Directory to save cropped faces

# Haar Cascade path (pre-trained face detection model)
face_cascade_path = cv2.data.haarcascades + "haarcascade_frontalface_default.xml"

# Load the Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(face_cascade_path)

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Function to resize an image to a fixed size (256x256)
def resize_to_fixed_size(image, size=(256, 256)):
    return cv2.resize(image, size, interpolation=cv2.INTER_AREA)

# Function to detect and crop faces from a single frame
def detect_and_crop_faces(frame_path, output_path):
    # Read the image
    image = cv2.imread(frame_path)
    if image is None:
        print(f"[ERROR] Failed to load image: {frame_path}")
        return 0

    # Convert to grayscale for face detection
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    face_count = len(faces)

    # If no faces are detected, return 0
    if face_count == 0:
        print(f"[INFO] No faces detected in {frame_path}")
        return 0

    print(f"[DEBUG] Detected {face_count} face(s) in {frame_path}")

    # Process each detected face
    for i, (x, y, w, h) in enumerate(faces):
        # Crop the face region
        face = image[y:y+h, x:x+w]

        # Resize the face to 256x256
        resized_face = resize_to_fixed_size(face)

        # Save the cropped and resized face
        face_filename = f"{os.path.splitext(os.path.basename(frame_path))[0]}_face_{i:02d}.jpg"
        face_output_path = os.path.join(output_path, face_filename)
        cv2.imwrite(face_output_path, resized_face)

    return face_count

# Traverse the extracted frames directory and process images
total_frames_processed = 0
total_faces_detected = 0

for root, dirs, files in os.walk(input_dir):
    for file in files:
        if file.endswith((".jpg", ".png")):  # Add other image formats if necessary
            frame_path = os.path.join(root, file)

            # Create a corresponding output directory for cropped faces
            relative_path = os.path.relpath(root, input_dir)
            frame_output_dir = os.path.join(output_dir, relative_path)
            os.makedirs(frame_output_dir, exist_ok=True)

            # Detect and crop faces
            faces_detected = detect_and_crop_faces(frame_path, frame_output_dir)

            # Update counts
            total_frames_processed += 1
            total_faces_detected += faces_detected

            # Debug message for each frame
            print(f"[DEBUG] Processed frame: {frame_path} | Faces detected: {faces_detected}")

# Final summary
print(f"[SUMMARY] Total frames processed: {total_frames_processed}")
print(f"[SUMMARY] Total faces detected: {total_faces_detected}")


[ERROR] Not enough frames to compute optical flow.
